In [1]:
# infer GPU in use
!nvidia-smi

Mon Jul 22 18:03:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   22C    P0              54W / 400W |      0MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import torch
import numpy as np
import os, sys
import matplotlib.pyplot as plt

# in this notebook, we will analyse the data files and try to see how we can make a custom data loader for the same.

data_y_s = np.load('/lcrc/project/NEXTGENOPT/NREL_COMSTOCK_DATA/grouped/G4601010_data.npz')
data_x_u = np.load('/lcrc/project/NEXTGENOPT/NREL_COMSTOCK_DATA/grouped/G4601010_weather.npz')

# function to calculate model sizes

def model_size_in_mb(model):
    param_size = sum(p.numel() * p.element_size() for p in model.parameters())
    size_in_mb = param_size / (1024 ** 2)
    return size_in_mb

In [3]:
# We now test out our dataset with california data

sys.path.insert(0,'/home/sbose/time-series-forecasting-federation')
from models.LFDataset import LFDataset

# create dataset
CA_dset = LFDataset(
    data_y_s = data_y_s,
    data_x_u = data_x_u,
    lookback = 8,
    lookahead = 4,
    client_idx = 0,
    idx_x = [0,1,2,3,4,5],
    idx_u = [6,7],
    dtype = torch.float32
)

# load into dataloader
from torch.utils.data import DataLoader
CA_dataloader = DataLoader(CA_dset, batch_size = 32, shuffle = True)

In [4]:
# Test out the shape of the dataloader outputs

for cidx, (a,b) in enumerate(CA_dataloader):
    print(f"On {cidx+1}th item of dataloader, type of a is {type(a)}, type of b is {type(b)}")
    for idx,itm in enumerate(a):
        print(f"Shape of {idx+1}th item in a is {itm.shape}.")
    for idx,itm in enumerate(b):
        print(f"Shape of {idx+1}th item in b is {itm.shape}.")
    break

On 1th item of dataloader, type of a is <class 'models.LFDataset.TensorList'>, type of b is <class 'models.LFDataset.TensorList'>
Shape of 1th item in a is torch.Size([32, 8, 1]).
Shape of 2th item in a is torch.Size([32, 8, 6]).
Shape of 3th item in a is torch.Size([32, 8, 2]).
Shape of 4th item in a is torch.Size([32, 8, 7]).
Shape of 5th item in a is torch.Size([32, 4, 2]).
Shape of 6th item in a is torch.Size([32, 4, 1]).
Shape of 1th item in b is torch.Size([32, 1]).
Shape of 2th item in b is torch.Size([32, 4, 1]).


In [5]:
# Ensure that relative imports from the git repository can always be found

import sys
sys.path.insert(0,'/home/sbose/time-series-forecasting-federation')

# kwargs for all models
model_kwargs = {
    'x_size': 6,
    'y_size': 1,
    'u_size': 2,
    's_size': 7,
    'lookback': 8,
    'lookahead': 4
}

In [6]:
# test out LSTM vanilla version

import torch
import torch.nn as nn
from models.LSTM.LSTMFCDecoder import LSTMFCDecoder

model = LSTMFCDecoder(
    **model_kwargs
)

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of LSTM FCNN output is {tuple(w.shape)}.")
    break

# print model size
print(f"LSTM FCNN head: {model_size_in_mb(model)} MB")

# create model of a different dtype
dtype, device = torch.float16, 'cuda'
model2 = LSTMFCDecoder(
    **model_kwargs,
    dtype=dtype
).to(device)

# evaluate the model
for a,b in CA_dataloader:
    a,b = a.to(dtype).to(device), b.to(dtype).to(device)
    w = model2(a)
    print(f"Input dtype is {dtype}.")
    print(f"Output dtype is device is {w.dtype}.")
    break

Shape of LSTM FCNN output is (32, 1).
LSTM FCNN head: 0.07667922973632812 MB
Input dtype is torch.float16.
Output dtype is device is torch.float16.


In [7]:
# test out LSTM autoregressive version

import torch
import torch.nn as nn
from models.LSTM.LSTMAR import LSTMAR

model = LSTMAR(
    **model_kwargs
)

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of LSTM AR output is {tuple(w.shape)}.")
    break

# print model size
print(f"LSTM AR: {model_size_in_mb(model)} MB")

# create model of a different dtype
dtype, device = torch.float16, 'cuda'
model2 = LSTMAR(
    **model_kwargs,
    dtype=dtype
).to(device)

# evaluate the model
for a,b in CA_dataloader:
    a,b = a.to(dtype).to(device), b.to(dtype).to(device)
    w = model2(a)
    print(f"Input dtype is {dtype}.")
    print(f"Output dtype is device is {w.dtype}.")
    break

Shape of LSTM AR output is (32, 4, 1).
LSTM AR: 0.045818328857421875 MB
Input dtype is torch.float16.
Output dtype is device is torch.float16.


In [8]:
# test out DARNN

import torch
import torch.nn as nn
from models.DARNN.DARNN import DARNN

model = DARNN(
    **model_kwargs
)

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of DARNN output is {tuple(w.shape)}.")
    break

# print model size
print(f"DARNN: {model_size_in_mb(model)} MB")

# create model of a different dtype
dtype, device = torch.float16, 'cuda'
model2 = DARNN(
    **model_kwargs,
    dtype=dtype
).to(device)

# evaluate the model
for a,b in CA_dataloader:
    a,b = a.to(dtype).to(device), b.to(dtype).to(device)
    w = model2(a)
    print(f"Input dtype is {dtype}.")
    print(f"Output dtype is device is {w.dtype}.")
    break

Shape of DARNN output is (32, 4, 1).
DARNN: 0.0562744140625 MB
Input dtype is torch.float16.
Output dtype is device is torch.float16.


In [9]:
# test out Transformer

import torch
import torch.nn as nn
from models.TRANSFORMER.TransformerAR import TransformerAR

model = TransformerAR(
    **model_kwargs
)

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    wtest = model(a,mode='test')
    print(f"Shape of Transformer AR output is {tuple(w.shape)}.")
    print(f"Shape of Transformer AR output in test mode is {tuple(wtest.shape)}.")
    break

# print model size
print(f"Transformer AR: {model_size_in_mb(model)} MB")

# create model of a different dtype
dtype, device = torch.float16, 'cuda'
model2 = TransformerAR(
    **model_kwargs,
    dtype=dtype
).to(device)

# evaluate the model
for a,b in CA_dataloader:
    a,b = a.to(dtype).to(device), b.to(dtype).to(device)
    w = model2(a)
    print(f"Input dtype is {dtype}.")
    print(f"Output dtype is device is {w.dtype}.")
    break

Shape of Transformer AR output is (32, 4, 1).
Shape of Transformer AR output in test mode is (32, 4, 1).
Transformer AR: 0.8964958190917969 MB
Input dtype is torch.float16.
Output dtype is device is torch.float16.


In [10]:
# test out Logtrans

import torch
import torch.nn as nn
from models.LOGTRANS.LogTransAR import LogTransAR
model = LogTransAR(
    **model_kwargs
)

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of Logtrans AR output is {tuple(w.shape)}.")
    break

# print model size
print(f"Logtrans AR: {model_size_in_mb(model)} MB")

# create model of a different dtype
dtype, device = torch.float16, 'cuda'
model2 = LogTransAR(
    **model_kwargs,
    dtype=dtype
).to(device)

# evaluate the model
for a,b in CA_dataloader:
    a,b = a.to(dtype).to(device), b.to(dtype).to(device)
    w = model2(a)
    print(f"Input dtype is {dtype}.")
    print(f"Output dtype is device is {w.dtype}.")
    break

Shape of Logtrans AR output is (32, 4, 1).
Logtrans AR: 0.7866249084472656 MB
Input dtype is torch.float16.
Output dtype is device is torch.float16.


In [11]:
# test out Informer

import torch
import torch.nn as nn
from models.INFORMER.Informer import Informer
model = Informer(
    **model_kwargs
)

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of Informer output is {tuple(w.shape)}.")
    break

# print model size
print(f"Informer: {model_size_in_mb(model)} MB")

# create model of a different dtype
dtype, device = torch.float16, 'cuda'
model2 = Informer(
    **model_kwargs,
    dtype=dtype
).to(device)

# evaluate the model
for a,b in CA_dataloader:
    a,b = a.to(dtype).to(device), b.to(dtype).to(device)
    w = model2(a)
    print(f"Input dtype is {dtype}.")
    print(f"Output dtype is device is {w.dtype}.")
    break

Shape of Informer output is (32, 4, 1).
Informer: 1.4672889709472656 MB
Input dtype is torch.float16.
Output dtype is device is torch.float16.


In [12]:
# test out Autoformer

import torch
import torch.nn as nn
from models.AUTOFORMER.Autoformer import Autoformer
model = Autoformer(
    **model_kwargs
)

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of Autoformer output is {tuple(w.shape)}.")
    break

# print model size
print(f"Autoformer: {model_size_in_mb(model)} MB")

# create model of a different dtype
dtype, device = torch.float16, 'cuda'
model2 = Autoformer(
    **model_kwargs,
    dtype=dtype
).to(device)

# evaluate the model
for a,b in CA_dataloader:
    a,b = a.to(dtype).to(device), b.to(dtype).to(device)
    w = model2(a)
    print(f"Input dtype is {dtype}.")
    print(f"Output dtype is device is {w.dtype}.")
    break

Shape of Autoformer output is (32, 4, 1).
Autoformer: 1.4076271057128906 MB


/home/sbose/time-series-forecasting-federation/models/AUTOFORMER/Autoformer.py:23: UserWarning: ComplexHalf support is experimental and many operators don't support it yet. (Triggered internally at /opt/conda/conda-bld/pytorch_1712608839953/work/aten/src/ATen/EmptyTensor.cpp:41.)
  rfft_output = torch.fft.rfft(padded_tensor, dim=-1)


Input dtype is torch.float16.
Output dtype is device is torch.float16.
